<a href="https://colab.research.google.com/github/Alfred9/Natural-Language-Processing/blob/main/Data%20Normalization/Text_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spark-nlp
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=85bef503b52d85ff97525901dd5a79fec5f113470e214ae852dd65b674ab4c4b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import sparknlp

# Start Spark Session
spark = sparknlp.start()


In [ ]:
# Import the required modules and classes
from sparknlp.base import DocumentAssembler, Pipeline, LightPipeline
from sparknlp.annotator import (
    Tokenizer,
    Normalizer
)
import pyspark.sql.functions as F

# Step 1: Transforms raw texts to `document` annotation
document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)

# Step 2: Gets the tokens of the text
tokenizer = (
    Tokenizer()
    .setInputCols(["document"])
    .setOutputCol("token")
)

# Step 3:  Normalizes the tokens
normalizer = (
    Normalizer()
    .setInputCols(["token"])
    .setOutputCol("normalized")
)

pipeline = Pipeline(stages=[document_assembler, tokenizer, normalizer])

In [ ]:
slang_text = """Hey, what's good fam? I'm just chillin with my squad.
                We're about to get to this dope party."""

#Create a Demo Slang CSV File
import csv

field_names = ['Slang', 'Correct_Word']

slangs = [
{'Slang': "fam", 'Correct_Word': 'friends'},
{'Slang': "chillin", 'Correct_Word': 'relaxing'},
{'Slang': "squad", 'Correct_Word': 'group of friends'},
{'Slang': "dope", 'Correct_Word': 'excellent'}
]

with open('slangs.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(slangs)

In [ ]:
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setSlangDictionary("/content/slangs.csv" ,",")


pipeline = Pipeline().setStages([
    document_assembler,
    tokenizer,
    normalizer
])

data = spark.createDataFrame([[slang_text]]) \
    .toDF("text")

model = pipeline.fit(data)
result = model.transform(data)
result.selectExpr("normalized.result").show(truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------+
|[Hey, whats, good, friends, Im, just, relaxing, with, my, group, of, friends, Were, about, to, get, to, this, excellent, party]|
+-------------------------------------------------------------------------------------------------------------------------------+

